In [152]:
import numpy as np
import pandas as pd
import math
from statsmodels import regression

In [153]:
data = pd.read_csv("../data/Case3HistoricalPrices.csv")
data = data.drop(columns=['Unnamed: 0'])
data.head(-5)

,S1,S2,S3,S4,S5,S6,S7,S8,B1,B2,...,B7,B8,C1,C2,C3,C4,C5,C6,C7,C8
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,98.355850,98.307641,95.630621,98.296988,98.474008,95.663069,98.342292,98.416837,100.086605,99.685232,...,99.674428,100.131478,95.854652,95.571202,94.085760,93.956485,94.140624,94.058865,95.786959,94.331430
2,96.766305,97.785655,95.211317,97.876191,96.910785,95.299426,97.862363,96.850881,100.595828,99.614514,...,99.463006,100.594669,87.500105,86.517650,88.861746,89.205952,89.255488,89.203200,86.873750,89.644765
3,99.498201,97.762932,95.831525,97.979482,99.992847,95.890271,97.890721,99.523488,100.687198,99.425275,...,99.220046,100.639175,84.500885,83.306741,86.183137,86.617107,86.643905,86.512592,83.788306,87.068357
4,96.259221,92.942010,94.272932,92.958011,96.469517,94.318577,93.007052,96.278064,101.939638,100.099827,...,99.843072,101.823484,79.974708,78.434282,80.585967,81.404902,81.484117,81.131945,79.157682,81.778226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,329.571663,332.038448,214.785331,268.624233,343.032460,110.231042,280.749048,451.478031,148.209537,145.528211,...,101.304242,164.288945,107.134821,86.988745,112.066514,59.802608,123.115258,165.989411,171.152575,93.702408
2511,333.241086,330.111672,212.581270,263.525020,347.112306,108.786471,279.410524,455.416775,148.666454,146.039146,...,101.227410,164.728097,107.284645,86.392955,112.446308,58.987521,122.068295,167.115678,170.716373,92.786356
2512,330.911166,327.267712,208.599873,258.850787,344.382675,109.801893,276.229074,453.069496,148.596464,145.782165,...,101.046012,164.217254,108.147948,86.707331,113.868806,59.554939,123.093553,168.137813,170.582716,92.188027
2513,329.808903,328.875381,207.622462,261.904975,343.104107,110.374945,276.415575,451.386406,148.861887,145.597568,...,100.763608,164.831889,107.793831,86.789811,113.321746,59.637080,123.042712,167.505370,170.763926,92.717958


In [154]:
train_split = 252*5 #8 Years
train = data.iloc[:train_split].reset_index(drop=True)
test = data.iloc[train_split:].reset_index(drop=True)

# Method 1:

Run OLS to get $b_0$ and $b_1$, where $b_0$ is the intercept and $b_1$ is the slope. Buy all stocks that have a positive $b_1$, and don't touch those assets.

In [4]:
# Initialize global variables
b1 = []
b0 = []
x = np.array([i for i in range(len(data))])
x = np.column_stack((x, [1]*len(x)))
for i in range(len(data.columns)):
    name = data.columns[i]
    prices = data[name].to_numpy()
    model = regression.linear_model.OLS(prices, x).fit()
    b1.append(model.params[0])
    b0.append(model.params[1])

def allocate1(asset_prices):
    portfolio = np.array([1 if b1[i] > 0 else 0 for i in range(len(b1))])
    if sum(portfolio): return portfolio / sum(portfolio) # Return normalized weights
    else: return portfolio

### Results

In [5]:
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = []

for index, row in test.iterrows():
    if index == 0: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate1(prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights))
        daily_return = sum(today) - sum(yesterday)
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate1(asset_prices)

In [6]:
total = 0
for i in range(1,6):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  0.9202531819092468
Sharpe ratio for year  2 :  1.5551529344237345
Sharpe ratio for year  3 :  0.15048333624809024
Sharpe ratio for year  4 :  1.0404178326300075
Sharpe ratio for year  5 :  -0.5528437461885013
Total points:  3.113463539022577


# Method 2:

Run OLS to get $b_0$ and $b_1$. Sell all stocks that are a standard deviation above the OLS price, and buy all stocks that are a standard deviation below the OLS price.

In [61]:
b = np.array([i if i>0 else 0 for i in b1])
b = b/sum(b)

In [62]:
b

array([0.08332675, 0.13616813, 0.07139314, 0.09039692, 0.09709472,
       0.        , 0.10922045, 0.11049149, 0.01936691, 0.00834448,
       0.01877392, 0.01239842, 0.02336239, 0.01272472, 0.        ,
       0.01959901, 0.025042  , 0.01844441, 0.02028844, 0.        ,
       0.01750195, 0.04299076, 0.05226642, 0.01080458])

Hyperparams for portfolio allocation based on b1: 1.8
Scale: 1.85

In [163]:
# Initialize global variables
b1 = []
b0 = []
std_err = []
x = np.array([i for i in range(len(data))])
x = np.column_stack((x, [1]*len(x)))
for i in range(len(data.columns)):
    name = data.columns[i]
    prices = data[name].to_numpy()
    model = regression.linear_model.OLS(prices, x).fit()
    b1.append(model.params[0])
    b0.append(model.params[1])
    line = x.T[0]*b1[i] + b0[i]
    adj_prices = prices - line
    std_err.append(np.std(adj_prices))
    
portfolio = np.array([i if i>0 else 0 for i in b1])
portfolio = portfolio/sum(portfolio)

t = len(train)+1
def allocate2(h, asset_prices):
    global t
    for i in range(len(asset_prices)):
        ols_price = t*b1[i] + b0[i]
        if asset_prices[i] > ols_price + h*std_err[i]:
            scale = 1
            portfolio[i] = max(portfolio[i] - scale, 0) # Sell
        elif asset_prices[i] < ols_price - h*std_err[i]:
            scale = 1
            portfolio[i] += scale # Buy
    t += 1
    if sum(portfolio): return portfolio / sum(portfolio) # Return normalized weights
    else: return portfolio

### Results

If you take out transation fee, the total points becomes positive

In [189]:
best_h = 1
best_sharpe = 0

for h in np.linspace(2.2, 2.4, 50):
    print(h)
    portfolio = np.array([i if i>0 else 0 for i in b1])
    portfolio = portfolio/sum(portfolio)

    t = len(train)+1
    # Annualized daily sharpe ratio
    weights = np.zeros(len(data.columns))
    prev_row = np.zeros(len(data.columns))
    returns = []

    for index, row in test.iterrows():
        if index == 0: # No returns on first day
            prev_row = row.to_numpy()
            new_weights = allocate2(h, prev_row)
        else: # Add daily return
            asset_prices = row.to_numpy()
            yesterday = prev_row * weights
            today = asset_prices * new_weights
            transaction_fee = 30*sum(abs(new_weights - weights)) 
            daily_return = sum(today) - sum(yesterday) - transaction_fee
            returns.append(daily_return)
            # Update prev_row and weights
            prev_row = asset_prices
            weights = new_weights
            new_weights = allocate2(h, asset_prices)

    total = 0
    for i in range(1,6):
        year_ret = np.array(returns[(i-1)*252: i*252])
        if year_ret.std():
            ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
        else: ratio = 0
        total += ratio
    if total > best_sharpe:
        best_sharpe = total
        best_h = h
#         print("Sharpe ratio for year ", i, ": ", ratio)
#     print("Total points: ", total)

2.2
2.2040816326530615
2.2081632653061227
2.212244897959184
2.2163265306122453
2.220408163265306
2.2244897959183674
2.2285714285714286
2.23265306122449
2.236734693877551
2.2408163265306125
2.2448979591836737
2.248979591836735
2.253061224489796
2.257142857142857
2.2612244897959184
2.2653061224489797
2.269387755102041
2.273469387755102
2.2775510204081635
2.2816326530612248
2.2857142857142856
2.289795918367347
2.293877551020408
2.2979591836734694
2.3020408163265307
2.306122448979592
2.3102040816326532
2.3142857142857145
2.3183673469387753
2.3224489795918366
2.326530612244898
2.330612244897959
2.3346938775510204
2.3387755102040817
2.342857142857143
2.3469387755102042
2.351020408163265
2.3551020408163263
2.3591836734693876
2.363265306122449
2.36734693877551
2.3714285714285714
2.3755102040816327
2.379591836734694
2.383673469387755
2.387755102040816
2.3918367346938774
2.3959183673469386
2.4


In [190]:
best_h, best_sharpe

(2.2612244897959184, 6.343312035583327)

Hyperparameter: 2.2612244897959184

In [176]:
portfolio = np.array([i if i>0 else 0 for i in b1])
portfolio = portfolio/sum(portfolio)

t = len(train)+1
# Annualized daily sharpe ratio
weights = np.zeros(len(data.columns))
prev_row = np.zeros(len(data.columns))
returns = []

for index, row in test.iterrows():
    if index == 0: # No returns on first day
        prev_row = row.to_numpy()
        new_weights = allocate2(2.2612244897959184, prev_row)
    else: # Add daily return
        asset_prices = row.to_numpy()
        yesterday = prev_row * weights
        today = asset_prices * new_weights
        transaction_fee = 30*sum(abs(new_weights - weights)) 
        daily_return = sum(today) - sum(yesterday) - transaction_fee
        returns.append(daily_return)
        # Update prev_row and weights
        prev_row = asset_prices
        weights = new_weights
        new_weights = allocate2(2.2612244897959184, asset_prices)

total = 0
for i in range(1,6):
    year_ret = np.array(returns[(i-1)*252: i*252])
    if year_ret.std():
        ratio = math.sqrt(252)*year_ret.mean()/year_ret.std()
    else: ratio = 0
    total += ratio
    print("Sharpe ratio for year ", i, ": ", ratio)
print("Total points: ", total)

Sharpe ratio for year  1 :  0.924203482754749
Sharpe ratio for year  2 :  -1.7121489620305637
Sharpe ratio for year  3 :  0.818607719533346
Sharpe ratio for year  4 :  1.3890231245120552
Sharpe ratio for year  5 :  4.886506712831402
Total points:  6.306192077600988
